In [0]:
#We have to define Spark Session to enter into Spark application
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.appname("VG_W47_Session").getorCreate()
 

In [0]:
%sql 
CREATE VOLUME IF NOT EXISTS bnbcatalog47.default.BnBVolume47;

In [0]:
rawdf1=spark.read.csv("/Volumes/bnbcatalog47/default/BnBVolume47/custsmodified",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
# rawdf1.show(20,False)
# display(rawdf1.take(20))


In [0]:
display(rawdf1.printSchema)
display(rawdf1.columns)
display(rawdf1.dtypes)

In [0]:
print(rawdf1.distinct().count())
print(rawdf1.distinct().count())
print(rawdf1.distinct().dropDuplicates(['id']).count())
print(rawdf1.describe())
print(rawdf1.schema)


In [0]:
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv("/Volumes/bnbcatalog47/default/BnBVolume47/",header=False,recursiveFileLookup=True,pathGlobFilter="custsmodified_N*" )
# display(rawdf1)

strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv("/Volumes/bnbcatalog47/default/BnBVolume47/",header=False,recursiveFileLookup=True,pathGlobFilter="custsmodified_T*" )
# display(rawdf2)
# rawdf3=rawdf1.union(rawdf2)
# rawdf3=rawdf1.unionByName(rawdf2,allowMissingColumns=True)
rawdf3=rawdf1.unionAll(rawdf2)
display(rawdf3)

In [0]:
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv("/Volumes/bnbcatalog47/default/BnBVolume47/",header=False,recursiveFileLookup=True,pathGlobFilter="custsmodified_N*" )
# display(rawdf1)
strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv("/Volumes/bnbcatalog47/default/BnBVolume47/",header=False,recursiveFileLookup=True,pathGlobFilter="custsmodified_T*" )

rawdf1.write.mode("overwrite").option("header","true").orc("/Volumes/bnbcatalog47/default/BnBVolume47/targetorc1")

rawdf2.write.mode("append").option("header","true").orc("/Volumes/bnbcatalog47/default/BnBVolume47/targetorc1")

scheve_df=spark.read.orc("/Volumes/bnbcatalog47/default/BnBVolume47/targetorc1",mergeSchema=True)
display(scheve_df)

In [0]:
from pyspark.sql.types import *
#Validation by doing cleansing
strt1="id int, firstname string, lastname string, age int, profession string"#applying schema strictly to validate and cleanse the data
strt11=StructType([StructField("id",IntegerType(),True),StructField("firstname",StringType(),True),StructField("lastname",StringType(),True),StructField("age",IntegerType(),True),StructField("profession",StringType(),True)])
dfmethod1=spark.read.schema(strt11).csv("/Volumes/bnbcatalog47/default/BnBVolume47/custsmodified",mode="PERMISSIVE",header=False)

 

display(dfmethod1)
print("entire count of data",dfmethod1.count())
print("after scrubbing, count of data",len(dfmethod1.collect()))


In [0]:
#method2 - drop malformed rows
strt1="id int, firstname string, lastname string, age int, profession string"#applying schema strictly to validate and 
dfmethod2=spark.read.schema(strt1).csv("/Volumes/bnbcatalog47/default/BnBVolume47/custsmodified",mode="dropMalformed",header=False)
#We are removing the entire row, where ever data format mismatch is there (number of columns, data type mismatch) (throwing away the entire potato or some portion of it by - cleansing)
display(dfmethod2)
print("entire count of data",dfmethod2.count())
print("after cleansing, count of data",len(dfmethod2.collect()))

In [0]:

from pyspark.sql.types import *
#Before actively Cleansing or Scrubbing - We have to create a Rejection Strategy to reduce data challenges in the future
strt11=StructType([StructField("id",IntegerType(),True),StructField("firstname",StringType(),True),StructField("lastname",StringType(),True),StructField("age",IntegerType(),True),StructField("profession",StringType(),True),StructField("corruptdata",StringType())])
dfmethod3=spark.read.schema(strt11).csv("/Volumes/bnbcatalog47/default/BnBVolume47/custsmodified",mode="PERMISSIVE",header=False,columnNameOfCorruptRecord="corruptdata")
# display(dfmethod3)
dfrejec1=dfmethod3.filter(dfmethod3.corruptdata.isNotNull())
display(dfrejec1)
dfrejec1.write.mode("overwrite").option("header","true").csv("/Volumes/bnbcatalog47/default/BnBVolume47/dfrejec1")

display(len(dfrejec1.collect()))


In [0]:
#We already know how to do cleansing applying the strict Structure on method1 and method2, but we can do cleansing and scrubbing in a controlled fashion by applying functions on the method3 dataframe
#Important na functions we can use to do cleansing

from pyspark.sql.types import *
strt11=StructType([StructField("id",IntegerType(),True),StructField("firstname",StringType(),True),StructField("lastname",StringType(),True),StructField("age",IntegerType(),True),StructField("profession",StringType(),True),StructField("corruptdata",StringType())])
dfmethod3=spark.read.schema(strt11).csv("/Volumes/bnbcatalog47/default/BnBVolume47/custsmodified",mode="PERMISSIVE",header=False,columnNameOfCorruptRecord="corruptdata")

# cleaningdf1=dfmethod3.na.drop(how="all")
# display(cleaningdf1)
cleaningdf1=dfmethod3.na.drop(how="any",subset=['id','profession'])
display(cleaningdf1)
# cleaningdf2=dfmethod3.na.drop(how="any")
# display(cleaningdf2)

In [0]:
#Before scrubbing, lets take the right cleansed data with id as null and entire row as null removed out
#Finally I am arriving for our current data, lets perform the best cleansing
cleansed_df=dfmethod3.na.drop(subset=["id"]).na.drop(how="all")
print("Final cleansed DF",len(cleansed_df.collect()))
display(cleansed_df.take(15))

In [0]:
scrubbed_df1=cleansed_df.na.fill("na",subset=["firstname","lastname"]).na.fill("not provided",subset=["profession"])
display(scrubbed_df1)

cleansed_df.na.replace("IT","Information Technology",subset=["profession"]).show()

dict1={"IT":"Information Technologies","Pilot":"Aircraft Pilot","Actor":"Celebrity"}
scrubbed_df=scrubbed_df1.na.replace(dict1,subset=["profession"])
display(scrubbed_df)

In [0]:
from pyspark.sql.functions import *
scrubbed_df=scrubbed_df.withColumn("sourcesystem",lit("BnBVolume47"))
display(scrubbed_df)

In [0]:
display(scrubbed_df.select("id","firstname","lastname","age","profession").na.drop(subset=["id"]).na.drop(how="all").na.drop(how="any"))

Stand_df1=scrubbed_df.select("id","firstname","lastname","age","profession").na.drop(subset=["id"]).na.drop(how="all").na.drop(how="any")
        
     

In [0]:
# display(Stand_df1.withColumnRenamed("id","Customer Id"))
# display(Stand_df1.where("id =4000000")) 
display(Stand_df1.orderBy(["id","firstname"],ascending=[True,False]).coalesce(1))

In [0]:
display(Stand_df1.withColumn("Customer id",col("id")))

In [0]:
# display(Stand_df1)
enrich_df6=Stand_df1.withColumn("profsplit",split(col("profession"),' '))
display(enrich_df6.withColumn("profirstcol",col("profsplit")[0]).withColumn("prosecondcol",col("profsplit")[size(col("profsplit"))-1]))
# display(enrich_df6)